# Export a model with control flow to ONNX

## Overview
This tutorial demonstrates how to handle control flow logic while exporting a PyTorch model to ONNX. It highlights the challenges of exporting conditional statements directly and provides solutions to circumvent them.

Conditional logic cannot be exported into ONNX unless they refactored to use torch.cond(). Let’s start with a simple model implementing a test.

What you will learn:

How to refactor the model to use torch.cond() for exporting.

How to export a model with control flow logic to ONNX.

How to optimize the exported model using the ONNX optimizer.

## Prerequisites
torch >= 2.6



In [1]:
import torch 

## Define the Models
Two models are defined:

ForwardWithControlFlowTest: A model with a forward method containing an if-else conditional.

ModelWithControlFlowTest: A model that incorporates ForwardWithControlFlowTest as part of a simple MLP. The models are tested with a random input tensor to confirm they execute as expected.

In [2]:
class ForwardWithControlFlowTest(torch.nn.Module):
    def forward(self, x):
        if x.sum():
            return x * 2
        return -x


class ModelWithControlFlowTest(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(3, 2),
            torch.nn.Linear(2, 1),
            ForwardWithControlFlowTest(),
        )

    def forward(self, x):
        out = self.mlp(x)
        return out


model = ModelWithControlFlowTest()

## Exporting the Model: First Attempt
Exporting this model using torch.export.export fails because the control flow logic in the forward pass creates a graph break that the exporter cannot handle. This behavior is expected, as conditional logic not written using torch.cond() is unsupported.

A try-except block is used to capture the expected failure during the export process. If the export unexpectedly succeeds, an AssertionError is raised.

In [3]:
x = torch.randn(3)
model(x)

try:
    torch.export.export(model, (x,), strict=False)
    raise AssertionError("This export should failed unless PyTorch now supports this model.")
except Exception as e:
    print(e)

W1014 20:31:51.061000 68294 site-packages/torch/fx/experimental/symbolic_shapes.py:7934] Unable to find user code corresponding to {u0}


Could not guard on data-dependent expression Eq(u0, 1) (unhinted: Eq(u0, 1)).  (Size-like symbols: none)

Caused by: (_export/non_strict_utils.py:1051 in __torch_function__)
For more information, run with TORCH_LOGS="dynamic"
For extended logs when we create symbols, also add TORCHDYNAMO_EXTENDED_DEBUG_CREATE_SYMBOL="u0"
If you suspect the guard was triggered from C++, add TORCHDYNAMO_EXTENDED_DEBUG_CPP=1
For more debugging help, see https://docs.google.com/document/d/1HSuTTVvYH1pTew89Rtpeu84Ht3nQEFTYhAX3Ypa_xJs/edit?usp=sharing

For C++ stack trace, run with TORCHDYNAMO_EXTENDED_DEBUG_CPP=1

The following call raised this error:
  File "/var/folders/p_/7g5qyc8d5zv5nfywm65yhpkc0000gn/T/ipykernel_68294/3521990874.py", line 3, in forward
    if x.sum():


The error above occurred when calling torch.export.export. If you would like to view some more information about this error, and get a list of all other errors that may occur in your export call, you can replace your `export()` call wit




def forward(self, arg0_1: "f32[2, 3]", arg1_1: "f32[2]", arg2_1: "f32[1, 2]", arg3_1: "f32[1]", arg4_1: "f32[3]"):
     # File: /opt/anaconda3/envs/torch/lib/python3.12/site-packages/torch/nn/modules/linear.py:125 in forward, code: return F.linear(input, self.weight, self.bias)
    linear: "f32[2]" = torch.ops.aten.linear.default(arg4_1, arg0_1, arg1_1);  arg4_1 = arg0_1 = arg1_1 = None
    linear_1: "f32[1]" = torch.ops.aten.linear.default(linear, arg2_1, arg3_1);  linear = arg2_1 = arg3_1 = None
    
     # File: /var/folders/p_/7g5qyc8d5zv5nfywm65yhpkc0000gn/T/ipykernel_68294/3521990874.py:3 in forward, code: if x.sum():
    sum_1: "f32[]" = torch.ops.aten.sum.default(linear_1);  linear_1 = None
    ne: "b8[]" = torch.ops.aten.ne.Scalar(sum_1, 0);  sum_1 = None
    item: "Sym(Eq(u0, 1))" = torch.ops.aten.item.default(ne);  ne = item = None
    



def forward(self, arg0_1: "f32[2, 3]", arg1_1: "f32[2]", arg2_1: "f32[1, 2]", arg3_1: "f32[1]", arg4_1: "f32[3]"):
     # File: /opt/a

## Suggested Patch: Refactoring with torch.cond()
To make the control flow exportable, the tutorial demonstrates replacing the forward method in ForwardWithControlFlowTest with a refactored version that uses torch.cond`().

Details of the Refactoring:

Two helper functions (identity2 and neg) represent the branches of the conditional logic: * torch.cond`() is used to specify the condition and the two branches along with the input arguments. * The updated forward method is then dynamically assigned to the ForwardWithControlFlowTest instance within the model. A list of submodules is printed to confirm the replacement.

In [4]:
def new_forward(x):
    def identity2(x):
        return x * 2

    def neg(x):
        return -x

    return torch.cond(x.sum() > 0, identity2, neg, (x,))


print("the list of submodules")
for name, mod in model.named_modules():
    print(name, type(mod))
    if isinstance(mod, ForwardWithControlFlowTest):
        mod.forward = new_forward

the list of submodules
 <class '__main__.ModelWithControlFlowTest'>
mlp <class 'torch.nn.modules.container.Sequential'>
mlp.0 <class 'torch.nn.modules.linear.Linear'>
mlp.1 <class 'torch.nn.modules.linear.Linear'>
mlp.2 <class '__main__.ForwardWithControlFlowTest'>


Let’s see what the FX graph looks like.

In [5]:
print(torch.export.export(model, (x,), strict=False))

ExportedProgram:
    class GraphModule(torch.nn.Module):
        def forward(self, p_mlp_0_weight: "f32[2, 3]", p_mlp_0_bias: "f32[2]", p_mlp_1_weight: "f32[1, 2]", p_mlp_1_bias: "f32[1]", x: "f32[3]"):
             # File: /opt/anaconda3/envs/torch/lib/python3.12/site-packages/torch/nn/modules/linear.py:125 in forward, code: return F.linear(input, self.weight, self.bias)
            linear: "f32[2]" = torch.ops.aten.linear.default(x, p_mlp_0_weight, p_mlp_0_bias);  x = p_mlp_0_weight = p_mlp_0_bias = None
            linear_1: "f32[1]" = torch.ops.aten.linear.default(linear, p_mlp_1_weight, p_mlp_1_bias);  linear = p_mlp_1_weight = p_mlp_1_bias = None
            
             # File: /opt/anaconda3/envs/torch/lib/python3.12/site-packages/torch/nn/modules/container.py:244 in forward, code: input = module(input)
            sum_1: "f32[]" = torch.ops.aten.sum.default(linear_1)
            gt: "b8[]" = torch.ops.aten.gt.Scalar(sum_1, 0);  sum_1 = None
            
             # File: <

Let’s export again.

In [6]:
onnx_program = torch.onnx.export(model, (x,), dynamo=True)
print(onnx_program.model)

[torch.onnx] Obtain model graph for `ModelWithControlFlowTest([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ModelWithControlFlowTest([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
<
    ir_version=10,
    opset_imports={'': 18},
    producer_name='pytorch',
    producer_version='2.8.0',
    domain=None,
    model_version=None,
>
graph(
    name=main_graph,
    inputs=(
        %"x"<FLOAT,[3]>
    ),
    outputs=(
        %"getitem"<FLOAT,[1]>
    ),
    initializers=(
        %"mlp.0.bias"<FLOAT,[2]>{TorchTensor<FLOAT,[2]>(Parameter containing: tensor([ 0.4647, -0.0532], requires_grad=True), name='mlp.0.bias')},
        %"mlp.1.bias"<FLOAT,[1]>{TorchTensor<FLOAT,[1]>(Parameter containing: tensor([-0.0816], requires_grad=True), name='mlp.1.bias')},
        %"val_0"<FLO

We can optimize the model and get rid of the model local functions created to capture the control flow branches.

In [7]:
onnx_program.optimize()
print(onnx_program.model)

<
    ir_version=10,
    opset_imports={'': 18},
    producer_name='pytorch',
    producer_version='2.8.0',
    domain=None,
    model_version=None,
>
graph(
    name=main_graph,
    inputs=(
        %"x"<FLOAT,[3]>
    ),
    outputs=(
        %"getitem"<FLOAT,[1]>
    ),
    initializers=(
        %"mlp.0.bias"<FLOAT,[2]>{TorchTensor<FLOAT,[2]>(Parameter containing: tensor([ 0.4647, -0.0532], requires_grad=True), name='mlp.0.bias')},
        %"mlp.1.bias"<FLOAT,[1]>{TorchTensor<FLOAT,[1]>(Parameter containing: tensor([-0.0816], requires_grad=True), name='mlp.1.bias')},
        %"val_0"<FLOAT,[3,2]>{Tensor<FLOAT,[3,2]>(array([[-0.54519   , -0.38536644], [ 0.4633441 , -0.31098711], [-0.5663341 ,  0.1554555 ]], dtype=float32), name='val_0')},
        %"val_2"<FLOAT,[2,1]>{Tensor<FLOAT,[2,1]>(array([[ 0.4765927 ], [-0.01628318]], dtype=float32), name='val_2')},
        %"scalar_tensor_default"<FLOAT,[]>{Tensor<FLOAT,[]>(array(0., dtype=float32), name='scalar_tensor_default')},
        %"

## Conclusion
This tutorial demonstrates the challenges of exporting models with conditional logic to ONNX and presents a practical solution using torch.cond(). While the default exporters may fail or produce imperfect graphs, refactoring the model’s logic ensures compatibility and generates a faithful ONNX representation.

By understanding these techniques, we can overcome common pitfalls when working with control flow in PyTorch models and ensure smooth integration with ONNX workflows.

## Further reading
The list below refers to tutorials that ranges from basic examples to advanced scenarios, not necessarily in the order they are listed. Feel free to jump directly to specific topics of your interest or sit tight and have fun going through all of them to learn all there is about the ONNX exporter.

1. Exporting a PyTorch model to ONNX
2. Extending the ONNX exporter operator support
3. Export a model with control flow to ONNX